In [0]:
%run "./config"

In [0]:
spark.conf.set(
    "fs.azure.account.key.mystore1984.dfs.core.windows.net",
    f'{key}'
)

In [0]:
dbutils.widgets.text('Date','')
input_value= dbutils.widgets.get('Date')

In [0]:
df=spark.read.csv(f"{raw_folder_path}/salesorder.csv", header= True, inferSchema= True)

In [0]:
from pyspark.sql.functions import col,concat_ws, current_timestamp, lit, split,to_date
df_sales= df.withColumn('ModifiedDate', to_date(col('ModifiedDate')))\
               .drop('rowguid')\
               .withColumn("Date",lit(input_value))\
               .withColumnRenamed("ModifiedDate","Salesdate")


In [0]:
df_sales.show(2)

+------------+------------------+--------+---------+---------+-----------------+---------+----------+----+
|SalesOrderID|SalesOrderDetailID|OrderQty|ProductID|UnitPrice|UnitPriceDiscount|LineTotal| Salesdate|Date|
+------------+------------------+--------+---------+---------+-----------------+---------+----------+----+
|       71774|            110562|       1|      836|  356.898|              0.0|  356.898|2008-06-01|    |
|       71774|            110563|       1|      822|  356.898|              0.0|  356.898|2008-06-01|    |
+------------+------------------+--------+---------+---------+-----------------+---------+----------+----+
only showing top 2 rows


In [0]:
df_sales.printSchema()

root
 |-- SalesOrderID: integer (nullable = true)
 |-- SalesOrderDetailID: integer (nullable = true)
 |-- OrderQty: integer (nullable = true)
 |-- ProductID: integer (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- UnitPriceDiscount: double (nullable = true)
 |-- LineTotal: double (nullable = true)
 |-- Salesdate: date (nullable = true)
 |-- Date: string (nullable = false)



In [0]:
sales_data_processed= df_sales.write.mode( "overwrite").parquet(f"{processed_folder_path}/salesdata")